In [ ]:
# ====== 1️⃣ Install and import libraries ======
!pip install tensorflow opencv-python --quiet

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from google.colab import files
import zipfile

# ====== 2️⃣ Upload the zipped file ======
uploaded = files.upload()  # A button will appear to select the file (dataset.zip)

# ====== 3️⃣ Unzip the file ======
for fn in uploaded.keys():
    with zipfile.ZipFile(fn, "r") as zip_ref:
        zip_ref.extractall("dataset")  #  Create a folder dataset

# Check the files
print(os.listdir("dataset"))

# ====== 4️⃣ Prepare the images ======
IMG_SIZE = 128

def load_images(folder):
    images = []
    labels = []
    for label, subfolder in enumerate(['no','yes']):  # Here we renamed the folders
        path = os.path.join(folder, subfolder)
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

X, y = load_images('dataset')
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0  # Normalize

# ====== 5️⃣Split the data ======
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ====== 6️⃣ Data Augmentation ======
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# ====== 7️⃣ Build CNN model ======
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ====== 8️⃣Train the model ======
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=20)

# ====== 9️⃣ Evaluate the model ======
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

# ====== 🔟 Example: Predict on a new image ======
# Place a new image of the same size (128x128) in the Colab folder or upload it
# Example: 'new_mri.jpg'

# img = cv2.imread('new_mri.jpg', cv2.IMREAD_GRAYSCALE)
# img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
# img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1) / 255.0

# prediction = model.predict(img)
# if prediction[0][0] > 0.5:
#     print("MRI contains tumor")
# else:
#     print("MRI is normal")





Saving archive.zip to archive (1).zip
['no', 'brain_tumor_dataset', 'yes']


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,193 (12.60 MB)

 Trainable params: 3,304,193 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6016 - loss: 0.7519 - val_accuracy: 0.6078 - val_loss: 0.6239
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 919ms/step - accuracy: 0.5919 - loss: 0.6495 - val_accuracy: 0.6471 - val_loss: 0.5829
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.6663 - loss: 0.5856 - val_accuracy: 0.7255 - val_loss: 0.5326
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 926ms/step - accuracy: 0.6958 - loss: 0.5734 - val_accuracy: 0.7255 - val_loss: 0.5302
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7441 - loss: 0.5247 - val_accuracy: 0.7647 - val_loss: 0.5485
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 875ms/step - accuracy: 0.7907 - loss: 0.5221 - val_accuracy: 0.7843 - val_loss: 0.5553
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7290 - loss: 0.5974 - val_accuracy: 0.7647 - val_loss: 0.5437
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 871ms/step - accuracy: 0.8129 - loss: 0.5377 - val_accuracy: 0.7647 - val_loss: 0.5044
Epoc